In [72]:
import numpy as np
import os
import cv2
from collections import Counter
from PIL import Image

## Part 1

In [48]:
def parse_coordinates(file_path):
    coordinates = []

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            try:
                position_part, movement_part = line.split(' v=')
                position = tuple(map(float, position_part.replace('p=', '').split(',')))
                movement = tuple(map(float, movement_part.split(',')))
                coordinates.append({
                    'position': position,
                    'movement': movement
                })
            except ValueError as e:
                print(f"Error: {e}")

    return coordinates

def generate_grid(n, m):
    grid = np.zeros((n, m), dtype=float)
    return grid

In [ ]:
def is_inside_grid(grid, x, y):
    rows, cols = grid.shape
    return 0 <= x < rows and 0 <= y < cols

def calculate_diff_grid(grid, x, y):
    rows, cols = grid.shape

    if x < 0:
        adjusted_x = rows + x
    elif x >= rows:
        adjusted_x = x - rows 
    else:
        adjusted_x = x

    if y < 0:
        adjusted_y = cols + y
    elif y >= cols:
        adjusted_y = y - cols
    else:
        adjusted_y = y

    return adjusted_x, adjusted_y


def move_agent(grid, x, y, delta):
    dx, dy = delta
    nx, ny = x + dx, y + dy

    if is_inside_grid(grid, nx, ny):
        return nx, ny 
    else:
        return calculate_diff_grid(grid, nx, ny)

def simulate_agent(grid, start_x, start_y, delta, n_rounds):
    x, y = start_x, start_y

    for i in range(n_rounds):
        x, y = move_agent(grid, x, y, delta)

    return x, y


In [49]:
input_file = "input.txt"
width = 101
height = 103

coordinates = parse_coordinates(input_file)
grid = generate_grid(n=width, m=height)

positions = []

for agent in coordinates:
    position, delta = agent.values()
    end_position = simulate_agent(grid, position[0], position[1], delta, n_rounds=100)
    positions.append(tuple(end_position))

In [50]:
position_counter = Counter(positions)

In [45]:
def compute_quadrants(n, m, counter):
    matrix = np.zeros((n, m), dtype=int)
    
    for (x, y), value in counter.items():
        x = int(x)
        y = int(y)
        if 0 <= x < n and 0 <= y < m: 
            matrix[x, y] = value

    middle_row = n // 2
    middle_col = m // 2
    if n > 0:
        matrix[middle_row, :] = 0
    if m > 0:
        matrix[:, middle_col] = 0

    q1 = np.sum(matrix[:middle_row, middle_col+1:]) 
    q2 = np.sum(matrix[:middle_row, :middle_col]) 
    q3 = np.sum(matrix[middle_row+1:, :middle_col]) 
    q4 = np.sum(matrix[middle_row+1:, middle_col+1:])
    
    return q1 * q2 * q3 * q4


In [52]:
safety_factor = compute_quadrants(width, height, position_counter)
print(safety_factor)

215987200


## Part 2

In [68]:
def detect_christmas_tree(matrix):
    rows, cols = matrix.shape
    for row in range(rows - 2):  
        for col in range(1, cols - 1): 
            if (matrix[row, col] == 1 and
                matrix[row + 1, col - 1] == 1 and matrix[row + 1, col + 1] == 1 and
                matrix[row + 2, col - 2] == 1 and matrix[row + 2, col] == 1 and matrix[row + 2, col + 2] == 1):
                return True
    return False

In [87]:
output_dir = "day14_frames"
os.makedirs(output_dir, exist_ok=True)

In [85]:
video_file = "day14_video.avi"
fps = 2 
frame_size = (width, height)

# Initialize video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(video_file, fourcc, fps, (frame_size[1], frame_size[0]), isColor=False)

In [88]:
positions = [tuple(agent['position']) for agent in coordinates]
deltas = [tuple(agent['movement']) for agent in coordinates]

for i in range(10000):
    matrix = np.zeros((width, height), dtype=int)
    new_positions = []
    for j, delta in enumerate(deltas):
        x, y = positions[j]
        end_position = simulate_agent(grid, x, y, delta, n_rounds=1)
        new_x = int(end_position[0])
        new_y = int(end_position[1])
        matrix[new_x, new_y] = 1
        new_positions.append((new_x, new_y))
        
    im = Image.fromarray((matrix * 255).astype(np.uint8))
    im.save(os.path.join(output_dir, f"frame_{i}.png"))
    # if detect_christmas_tree(matrix):
    #     print(f"Tree detected after rounds {i}")
    #     im.show()
        
    frame = (matrix * 255).astype(np.uint8)
    video_writer.write(cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR))
    
    positions = new_positions
    
video_writer.release()